In [ ]:
## second run
from flask import Flask, request, jsonify, render_template
import numpy as np
import joblib
import os

app = Flask(__name__)

# Load the model and encoder
MODEL_PATH = 'random_forest_regressor.pkl'
ENCODER_PATH = 'encoder.pkl'

# Check if the model and encoder files exist
if not os.path.exists(MODEL_PATH) or not os.path.exists(ENCODER_PATH):
    raise FileNotFoundError("Model or Encoder file not found. Please ensure the files are available.")

random_forest_regressor = joblib.load(MODEL_PATH)
encoder = joblib.load(ENCODER_PATH)

@app.route('/')
def home():
    return render_template('login.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the data from the POST request
        data = request.get_json(force=True)
        
        # Validate input data
        if 'education_qualification' not in data or 'years_of_experience' not in data:
            return jsonify({'error': 'Invalid input data. Please provide education_qualification and years_of_experience.'}), 400
        
        # Preprocess the input data
        education_qualification = data['education_qualification']
        years_of_experience = data['years_of_experience']
        
        # Encode the education qualification
        education_encoded = encoder.transform([[education_qualification]]).toarray()
        
        # Create the input array
        input_array = np.array([years_of_experience] + education_encoded.tolist()[0]).reshape(1, -1)
        
        # Make the prediction using the Random Forest model
        prediction = random_forest_regressor.predict(input_array)
        
        # Return the prediction as a JSON response
        return jsonify({'predicted_salary': prediction[0]})
    except Exception as e:
        return jsonify({'error': str(e)}), 500


app.run()